In [1]:
import pandas as pd
import numpy as mp
import datetime
import matplotlib.pyplot as plt

In [ ]:
#dealt with old version of csv
import pandas as pd
from datetime import datetime


file = 'precisefinal.csv'
df = pd.read_csv(file)
df['bookid'] = df['bookid'].astype(int)
for i in df:
    print(i)
# file = 'precisefinal.csv'
# df = pd.read_csv(file)
df['bookid'] = df['bookid'].astype(int)
# df['clickTime'] = df['clickTime'].apply(lambda x: process(x))
df['clickTime'] = pd.to_datetime(df['clickTime'], format='%Y-%m-%d %H:%M:%S')


df['hs_name'] = df['hotspotName'].apply(lambda x: x.split('_')[0])
print(df['hs_name'])
hs_names = df['hs_name'].unique()

for name in hs_names:
    if name.startswith('p'):
        hs_types = ['quotes', 'vocabulary']
        for hs_type in hs_types:
            tmp = df.loc[(df['hs_name'] == name) & (df['hotspotType'] == hs_type)].copy()
            print(tmp.shape[0])
            if tmp.shape[0] > 1:
                tmp['diff'] = tmp['clickTime'].diff(1)
                tmp['diff'] = tmp['diff'].apply(lambda x: x.total_seconds())
                indexes = tmp.loc[(tmp['diff'] >= 0) & (tmp['diff'] <= 2)].index
                print(indexes)
                df = df.loc[~df.index.isin(indexes)]
                
for i in [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,17,18,23,29,
          30,31,43,45,50,56,57,59,61,64,65,71,72,73,79,81,
          83,90,97,100,102,108,114,115,116,121,125,126,
          128,13,131,132,134,136,138,140,143,145,146,147,
          150,152,154,158,159,160,161,163,164,
          167,168,174,184,199,200,201,209,213,215,217,
          239,243,251,253,255,257,259,265,267]:
    
    indexNames = df[ df['username'] == i ].index
    # Delete row indexes from dataFrame
    df.drop(indexNames , inplace=True)

df = df.sort_values(by=['username', 'opentime'])
df.to_csv('o.csv', index=False)




In [61]:

import pandas as pd


def process(time):
    time = time.split()
    time[0] = time[0].split('/')
    time[0][2] = '20' + time[0][2]
    time[0] = '/'.join(time[0])
    time = ' '.join(time)
    return time

def duration(x):
    #start time when user read the book(bookid ->0,1) 
    start_time = x.iloc[0, 3]

    #type of the discussion book
    type_ = x.iloc[0, 1]

    hotspot_name = x.iloc[:, 2]
    check = ''
    if 'p17_nextButton' in hotspot_name.tolist() or 'q17_nextButton' in hotspot_name.tolist():
        check = 'yes'
    else:
        check = 'no'

    q_dura = 0
    p_dura = 0
    end_time = start_time
    for i in range(1, x.shape[0]):
        click, name = x.iloc[i, 3], x.iloc[i, 1]
        if name == type_:
            end_time = click
        else:
            if type_ == 'q':
                q_dura =round(q_dura+((end_time - start_time).total_seconds())/60, 2)
            else:
                p_dura =round(p_dura+((end_time - start_time).total_seconds())/60, 2)
            start_time = click
            end_time = click
            type_ = name
    if type_ == 'q':
        q_dura =round(q_dura+((end_time - start_time).total_seconds())/60, 2)
    else:
        p_dura =round(p_dura+((end_time - start_time).total_seconds())/60, 2)
    return q_dura, p_dura, check


file = 'newwww_changed.csv'
df = pd.read_csv(file)
df['bookid'] = df['bookid'].astype(int)


# df['clickTime'] = df['clickTime'].apply(lambda x: process(x))
df['clickTime'] = pd.to_datetime(df['clickTime'], format='%Y/%m/%d %H:%M:%S')


df['date'] = df['clickTime'].apply(lambda x: x.strftime('%Y/%m/%d'))


df['name'] = df['hotspotName'].apply(lambda x: x[0])

df = df[['username', 'name', 'hotspotName', 'clickTime', 'date', 'bookid']]

df = df.drop_duplicates()
for i in [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,17,18,23,29,30,31,43,45,50,56,57,59,61,64,65,71,72,73,79,81,
          83,90,97,100,102,108,114,115,116,121,125,126,
          128,13,131,132,134,136,138,140,143,145,146,147,
          150,152,154,158,159,160,161,163,164,227,73,47,144,88,167,168,174,184,199,200,201,209,213,215,217,
          239,243,251,253,255,257,259,265,267]:
          
    indexNames = df[ df['username'] == i ].index
    # Delete row indexes from dataFrame
    df.drop(indexNames , inplace=True)
    
    
book_id = []
username = []
q_duration = []
p_duration = []
dates = []
checks = []

for user in df['username'].unique():
    unique_dates = df.loc[df['username'] == user]['date'].unique()
    for date in unique_dates:
        tmp = df.loc[(df['username'] == user) & (df['date'] == date)]
        q, p, check = duration(tmp)
        username.append(user)
        q_duration.append(q)
        p_duration.append(p)
        dates.append(date)
        checks.append(check)
        book_id.append(tmp['bookid'].iloc[0])

result = pd.DataFrame({'username': username, 'Q Clicktime duration': q_duration,
                       'P Clicktime duration': p_duration, 'Date': dates, 'Bookid': book_id, 'Check': checks})



result.to_csv('time.csv', index=False)

#from google.colab import files
#uploaded = files.upload()
#import io
#df1=pd.read_csv(io.BytesIO(uploaded['hotspotResult425.csv']))

In [62]:

df = pd.read_csv('time.csv')

participants = []
r1q = []
r1p = []
r1 = []
r2q = []
r2p = []
r2 = []
for user in df['username'].unique():
    tmp = df.loc[df['username'] == user]
    participants.append(user)
    bookids = tmp['Bookid'].unique()
    
    if bookids.shape[0] == 1:
#         print(bookids.shape[0])
        if bookids[0] == 0:
            r2q.append(0)
            r2p.append(0)
            r2.append(0)
        else:
            r1q.append(0)
            r1p.append(0)
            r1.append(0)

    for bookid in bookids:
        if bookid == 0:
            q_durations = tmp.loc[tmp['Bookid'] == bookid]['Q Clicktime duration'].sum()
            p_durations = tmp.loc[tmp['Bookid'] == bookid]['P Clicktime duration'].sum()
            r1q.append(q_durations)
            r1p.append(p_durations)
            r1.append(round(q_durations + p_durations,2))
        else:
            q_durations = tmp.loc[tmp['Bookid'] == bookid]['Q Clicktime duration'].sum()
            p_durations = tmp.loc[tmp['Bookid'] == bookid]['P Clicktime duration'].sum()
            r2q.append(q_durations)
            r2p.append(p_durations)
            r2.append(round(q_durations + p_durations,2))

# res = pd.DataFrame({'username': participants, 'R1Q_READ': r1q, 'R1P_READ': r1p, 'R1_READ': r1,
#                     'R2Q_READ': r2q, 'R2P_READ': r2p, 'R2_READ': r2})
# res['T_READ'] = round(res['R1_READ'] + res['R2_READ'],2)
# res.to_csv('lit.csv', index=False)
res = pd.DataFrame({'username': participants, 'discstime_r1': r1q, 'readtime_r1': r1p, 'R1_total': r1,
                    'discstime_r2': r2q, 'readtime_r2': r2p, 'R2_total': r2})
res['TOTOL_READ'] = round(res['R1_total'] + res['R2_total'],2)
res.to_csv('lit.csv', index=False)

In [63]:
from dateutil import parser

df = pd.read_csv('newwww_changed.csv')
df2 = pd.read_csv('newwww_changed.csv')
df2.drop(df.index, inplace=True)
indexDate = parser.parse("Aug 28 1980")
indexUser = 9999999999
for row in df.iterrows():
    date = parser.parse(row[1][2]).date()
    if date != indexDate and row[1][1] != indexUser:
#         print(indexUser)
#         print(indexDate)
#         print(row[1][1])
#         print(date)
        indexUser = row[1][1]
        indexDate = date
        df2 = df2.append(row[1])
    elif date == indexDate and row[1][1] == indexUser:
        df2 = df2.append(row[1])

df2.to_csv("1.csv",index=False)


with open('1.csv', 'r') as t1, open('newwww_changed.csv', 'r') as t2:
    fileone = t1.readlines()
    filetwo = t2.readlines()

with open('2.csv', 'w') as outFile:
    for line in filetwo:
        if line not in fileone:
            outFile.write(line)
df = pd.read_csv('2.csv')         
df.to_csv("2.csv", header=['bookid','username',	'opentime',	'closetime','hotspotName','hotspotType','clickResult','clickTime','remarks','hs_name'], index=False)



Correctness+Question Answered 

In [64]:
#必要的package
import pandas as pd
import numpy as mp
import datetime
import matplotlib.pyplot as plt

In [65]:
#导入文件
import pandas as pd

df1 = pd.read_csv ("1.csv")


In [66]:
#拿出userId
userList = []
for i in df1["username"]:
    if i not in userList:
        userList.append(i)

In [67]:
#新建两个空的dic, dic1用来放每个学生 实际做了几个选择题的数量， dic2用来放每个学生 第一次就选择了正确的答案的数量
dic_1={}
dic_2={}


for i in userList:
    dic_1[i]=0
    dic_2[i]=0
    #把每个学生所有的数据分别拿出来
    student0 = df1[df1.username == int(i)]
    #新建空的list2用于后面放题号
    list2=[]
    #新建count_right 用于后面计算第一次就选择了正确答案的数量
    count_right=0
    for index, row in student0.iterrows():
        #把每一行拿出来
        a=row
        #如果是discussion类型，就代表是多选题
        if a.hotspotType=="discussion":
            #name 是题号
            name=a.hotspotName
            name=str(name)[1:]
            #如果题号没有被记录
            if name not in list2:
                #把题号放进list
                list2.append(name)
                #看第一遍result 是否是对的，对的就次数加一
                result=str(a.clickResult)
                if result.endswith("answer"):
                    count_right+=1
    dic_1[i]=len(list2)
    dic_2[i]=count_right

In [68]:
#算正确率， 保留三位小数
dic_rate={}
for i in userList:
    i=int(i)
    #判断 实际选择题次数是否为0， 为0的话直接算0
    if dic_1[i]==0:
        dic_rate[i]=0
    else:
        #用正确率/次数， 保留三位小数
        dic_rate[i]=round(dic_2[i]/dic_1[i],3)

In [69]:
#a是userID, b是每个学生实际做了几个选择题的数量， c是每个学生第一次就选择了正确的答案的数量, d是c/b-Right Rate
a=userList
b=list(dic_1.values())
c=list(dic_2.values())
d=list(dic_rate.values())

In [70]:
#改成dataframe格式
df3 = pd.DataFrame({'UserName' : a,'NumQuestionAnswer':b, 'NumRightFirstly' : c,'RightRate':d})

In [73]:
#导出文件
df3.to_csv('RightRate1.csv')

In [18]:
import pandas as pd
df1 = pd.read_csv ("RightRate1.csv")
df2 = pd.read_csv ("RightRate2.csv")


df =pd.merge(df1, df2, how='inner', on=['UserName'])
df.to_csv('df.csv')



In [2]:
import csv
import pandas as pd

final = []
with open("time.csv") as f:
    
    reader = csv.reader(f)
    i =0
    for row in reader:
        if i % 2 == 0:
            newRow = row
            
        else:
            newRow = newRow + row
#             print(newRow)
            
            final.append(newRow)
        i = i + 1
head = ['username',	'Q1 Clicktime duration',	'P1 Clicktime duration',	'Date1',	'Bookid1','Check1',
       'username2',	'Q2 Clicktime duration',	'P2 Clicktime duration',	'Date2',	'Bookid2','Check2']  
with open("time_update_file.csv",'w') as f1:
      
    write = csv.writer(f1)
    write.writerow(head)
    write.writerows(final)
    

In [11]:
# columns_to_be_removed = ['username2']

# data = pd.read_csv('time_update_file.csv').drop(columns_to_be_removed, axis = 'columns')


# # keep_col =['username',	'Q1 Clicktime duration','P1 Clicktime duration','Date1',	'Bookid1','Check1',
# #      'Q2 Clicktime duration',	'P2 Clicktime duration',	'Date2',	'Bookid2','Check2']  
# # new_f.to_csv("time_date_check_info.csv", index=False)

In [17]:
df1 = pd.read_csv ("time_update_file.csv")
df2 = pd.read_csv ("lit.csv")


df =pd.merge(df1, df2 ,on=['username'])
df.to_csv('dfinal.csv')



In [30]:
df = pd.read_csv ("ordf.csv")

# df = df.drop_duplicates()
# for i in [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,17,18,23,29,30,31,43,45,50,56,57,59,61,64,65,71,72,73,79,81,
#           83,90,97,100,102,108,114,115,116,121,125,126,
#           128,13,131,132,134,136,138,140,143,145,146,147,
#           150,152,154,158,159,160,161,163,164,
#           167,168,174,184,199,200,201,209,213,215,217,
#           239,243,251,253,255,257,259,265,267,   227,73,47,144,88]:
          
#     indexNames = df[ df['username'] == i ].index
#     df.drop(indexNames , inplace=True)
#     indexNames = df[ df['username'] == 227 ].index
#     df.drop(indexNames , inplace=True)


In [24]:
# i = [16,19,20,21,22,24,25,26,27,28,32,33,34,35,36,37,38,39,40,41,42,44,46,48,49,51,52,53,54,55,60,62,63,66,67,68,69,70,74,75,76,77,78,80,82,84,85,86,87,89,91,92,94,95,99,101,103,104,105,106,107,109,110,111,112,113,117,118,119,120,122,123,124,127,129,130,133,135,137,139,141,142,148,149,151,153,155,156,157,162,165,166,169,171,173,175,177,179,181,183,185,187,188,189,191,193,195,196,197,202,203,205,207,211,219,221,223,225,229,231,233,235,237,241,245,247,249,261,263,269,271]
# o=  [16,19,20,21,22,24,25,26,27,28,32,33,34,35,36,37,38,39,40,41,42,44,46,47,48,49,51,52,53,54,55,60,62,63,66,67,68,69,70,73,74,75,76,77,78,80,82,84,85,86,87,88,89,91,92,94,95,99,101,103,104,105,106,107,109,110,111,112,113,117,118,119,120,122,123,124,127,129,130,133,135,137,139,141,142,144,149,151,153,155,156,157,162,165,166,169,171,173,175,177,179,181,183,185,187,188,189,191,193,195,196,197,202,203,205,207,211,219,221,223,225,227,229,231,233,235,237,241,245,247,249,261,263,269,271]
# print(list(set(o) - set(i)))

[227, 73, 47, 144, 88]
